In [ ]:
from astropy.io import fits

import numpy as np

from lmfit import Model

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 13})

In [ ]:
# Load COLA1's 1D spectrum
SPECTRA_FOLDER = '../../spectra/SPECTRA_O3_FINAL/ONED/'
dat = fits.open(f'{SPECTRA_FOLDER}/spectrum_1D_COLA1_9269.fits')
data_1d = dat[1].data
data_1d.columns

In [ ]:
# The models
def gaussian(x, totflux, c, x0, sigma):
    return totflux*((sigma)**-1 * (2*np.pi)**-0.5 * np.exp(-(x-x0)**2/(2*sigma**2)))+c


def double_gaussian(x, totflux, scale, narrow_x0, broad_x0,
                    narrow_sigma, broad_sigma):
    narrow_flux = scale * totflux
    broad_flux = (1 - scale) * totflux
    return gaussian(x, narrow_flux, 0., narrow_x0, narrow_sigma) \
        + gaussian(x, broad_flux, 0., narrow_x0, broad_sigma)

In [ ]:
# First fit the 5008 line to get the FWHM

this_z = 6.5916

O3_fit_params = {}

for module in ['A', 'B']:
    flux_tot = data_1d.field(f'flux_tot_{module}')
    flux_tot_err = data_1d.field(f'flux_tot_{module}_err')

    obs_wav = data_1d.field('wavelength')  # Angstroms

    # Define wavelength interval
    thisline = 5008.24
    sel_include = (obs_wav > 4975 * (1 + this_z)) * \
        (obs_wav < 5050 * (1 + this_z))

    model = Model(gaussian)
    model.set_param_hint('totflux', min=0., max=200)
    model.set_param_hint('sigma', min=0.1, max=50)
    model.set_param_hint('c', min=-0.005, max=0.005)
    model.set_param_hint('x0', min=(1 + this_z)*thisline - 3,
                         max=(1 + this_z)*thisline + 3)
    params = model.make_params(totflux=10, sigma=15.,
                               c=0, x0=(1+this_z)*thisline)

    result = model.fit(flux_tot[sel_include], x=obs_wav[sel_include],
                       params=params, weights=1. / flux_tot_err[sel_include],
                       nan_policy='propagate', method='differential_evolution',
                       max_nfev=100000)

    print(result.fit_report())

    # Plot the fit
    fig, ax = plt.subplots()

    ax.plot(obs_wav[sel_include], flux_tot[sel_include],
            color='tab:blue', drawstyle='steps-mid')
    ax.fill_between(obs_wav[sel_include], -flux_tot_err[sel_include],
                    flux_tot_err[sel_include], lw=0, alpha=0.4, color='tab:blue')

    xx = np.arange(min(obs_wav[sel_include]),
                   max(obs_wav[sel_include]), 0.01)
    ax.plot(xx, model.eval(result.params, x=xx),
            lw=2, color='k', alpha=0.5)

    plt.show()

    # Save the FWHM
    O3_fit_params[f'sigma_{module}'] = result.params['sigma'].value
    O3_fit_params[f'sigma_{module}_err'] = result.params['sigma'].stderr
    O3_fit_params[f'totflux_{module}'] = result.params['totflux'].value
    O3_fit_params[f'totflux_{module}_err'] = result.params['totflux'].stderr
    O3_fit_params[f'c_{module}'] = result.params['c'].value
    O3_fit_params[f'c_{module}_err'] = result.params['c'].stderr

In [ ]:
# Now fit Hbeta with two components
for module in ['A', 'B']:
    flux_tot = data_1d.field(f'flux_tot_{module}')
    flux_tot_err = data_1d.field(f'flux_tot_{module}_err')

    obs_wav = data_1d.field('wavelength')  # Angstroms

    # Define wavelength interval
    thisline = 4862.68
    sel_include = (obs_wav > 4830*(1 + this_z))*(obs_wav < 4890*(1 + this_z))

    model = Model(double_gaussian)

    model.set_param_hint('totflux', min=0.1, max=10)
    model.set_param_hint(
        'c', value=O3_fit_params[f'c_{module}'] * 0.5, vary=False)
    model.set_param_hint('narrow_sigma',
                         min=O3_fit_params[f'sigma_{module}'] - O3_fit_params[f'sigma_{module}_err'],
                         max=O3_fit_params[f'sigma_{module}'] + O3_fit_params[f'sigma_{module}_err'] + 0.001)
    model.set_param_hint('broad_sigma', min=O3_fit_params[f'sigma_{module}'],
                         max=200.)
    model.set_param_hint('narrow_x0', min=(1 + this_z)*thisline - 10,
                         max=(1 + this_z)*thisline + 10)
    model.set_param_hint('broad_x0', min=(1 + this_z)*thisline - 10,
                         max=(1 + this_z)*thisline + 10, vary=False)
    model.set_param_hint('scale', min=0.1, max=1.)

    params = model.make_params(broad_sigma=30.)

    for try_i in range(5):
        print(f'Try: {try_i + 1}')
        result = model.fit(flux_tot[sel_include], x=obs_wav[sel_include],
                           params=params, weights=1. /
                           flux_tot_err[sel_include],
                           nan_policy='propagate', method='differential_evolution',
                           max_nfev=100000)
        good_fit = result.errorbars
        if not good_fit:
            continue

    print(result.fit_report())

    # Plot the fit
    fig, ax = plt.subplots()

    ax.plot(obs_wav[sel_include], flux_tot[sel_include],
            color='tab:blue', drawstyle='steps-mid')
    ax.fill_between(obs_wav[sel_include], -flux_tot_err[sel_include],
                    flux_tot_err[sel_include], lw=0, alpha=0.4, color='tab:blue')

    xx = np.arange(min(obs_wav[sel_include]),
                   max(obs_wav[sel_include]), 0.01)
    ax.plot(xx, gaussian(xx, result.params['totflux'] * result.params['scale'].value,
                         O3_fit_params[f'c_{module}'] * 0,
                         result.params['narrow_x0'].value,
                         result.params['narrow_sigma'].value),
            lw=2, color='r', alpha=0.7, label='Narrow')
    ax.plot(xx, gaussian(xx, result.params['totflux'] * (1 - result.params['scale'].value),
                         O3_fit_params[f'c_{module}'] * 0,
                         result.params['narrow_x0'].value,
                         result.params['broad_sigma'].value),
            lw=2, color='b', alpha=0.7, label='Broad')

    ax.plot(xx, model.eval(result.params, x=xx),
            lw=2, color='k', label='Total', ls='--')

    ax.legend()
    ax.set_title(f'Module {module}')

    plt.show()

In [ ]:
# First fit the 5008 line to get the FWHM

this_z = 6.5916

O3_fit_params = {}

flux_tot_A = data_1d.field(f'flux_tot_A')
flux_tot_A_err = data_1d.field(f'flux_tot_A_err')
flux_tot_B = data_1d.field(f'flux_tot_B')
flux_tot_B_err = data_1d.field(f'flux_tot_B_err')

flux_tot = (flux_tot_A * flux_tot_A_err**-2 + flux_tot_B * flux_tot_B_err**-2)\
    / (flux_tot_A_err**-2 + flux_tot_B_err**-2)
flux_tot_err = (flux_tot_A_err**-2 + flux_tot_B_err**-2) ** -0.5

obs_wav = data_1d.field('wavelength')  # Angstroms

# Define wavelength interval
thisline = 5008.24
sel_include = (obs_wav > 4975 * (1 + this_z)) * (obs_wav < 5050 * (1 + this_z))

model = Model(gaussian)
model.set_param_hint('totflux', min=0., max=200)
model.set_param_hint('sigma', min=0.1, max=50)
model.set_param_hint('c', min=-0.005, max=0.005)
model.set_param_hint('x0', min=(1 + this_z)*thisline - 3,
                     max=(1 + this_z)*thisline + 3)
params = model.make_params(totflux=10, sigma=15.,
                           c=0, x0=(1+this_z)*thisline)

result = model.fit(flux_tot[sel_include], x=obs_wav[sel_include],
                   params=params, weights=1. / flux_tot_err[sel_include],
                   nan_policy='propagate', method='differential_evolution',
                   max_nfev=100000)

print(result.fit_report())

# Plot the fit
fig, ax = plt.subplots()

ax.plot(obs_wav[sel_include], flux_tot[sel_include],
        color='tab:blue', drawstyle='steps-mid')
ax.fill_between(obs_wav[sel_include], -flux_tot_err[sel_include],
                flux_tot_err[sel_include], lw=0, alpha=0.4, color='tab:blue')

xx = np.arange(min(obs_wav[sel_include]),
               max(obs_wav[sel_include]), 0.01)
ax.plot(xx, model.eval(result.params, x=xx),
        lw=2, color='k', alpha=0.5)

plt.show()

# Save the FWHM
sigma = result.params['sigma'].value
sigma_err = result.params['sigma'].stderr
totflux = result.params['totflux'].value
totflux_err = result.params['totflux'].stderr
c = result.params['c'].value
c_err = result.params['c'].stderr

In [ ]:
# Define wavelength interval
thisline = 4862.68
sel_include = (obs_wav > 4830*(1 + this_z))*(obs_wav < 4890*(1 + this_z))

model = Model(double_gaussian)

model.set_param_hint('totflux', min=0.1, max=10)
model.set_param_hint('c', value=c * 0.5, vary=False)
model.set_param_hint('narrow_sigma',
                     min=sigma - sigma_err*0.03,
                     max=sigma + sigma_err*0.03)
model.set_param_hint('broad_sigma',
                #      min=52.29,
                     min=sigma - sigma_err*3,
                     max=200.)
model.set_param_hint('narrow_x0', min=(1 + this_z)*thisline - 10,
                     max=(1 + this_z)*thisline + 10)
model.set_param_hint('broad_x0', min=(1 + this_z)*thisline - 10,
                     max=(1 + this_z)*thisline + 10, vary=False)
model.set_param_hint('scale', min=0.1, max=1.)

params = model.make_params(broad_sigma=30., narrow_sigma=sigma, scale=0.9)

for try_i in range(5):
    print(f'Try: {try_i + 1}')
    result = model.fit(flux_tot[sel_include], x=obs_wav[sel_include],
                       params=params, weights=1. / flux_tot_err[sel_include],
                       nan_policy='propagate', method='differential_evolution',
                       max_nfev=100000)
    good_fit = result.errorbars
    if not good_fit:
        continue

print(result.fit_report())

# Plot the fit
fig, ax = plt.subplots()

ax.plot(obs_wav[sel_include], flux_tot[sel_include],
        color='tab:blue', drawstyle='steps-mid')
ax.fill_between(obs_wav[sel_include], -flux_tot_err[sel_include],
                flux_tot_err[sel_include], lw=0, alpha=0.4, color='tab:blue')

xx = np.arange(min(obs_wav[sel_include]),
               max(obs_wav[sel_include]), 0.01)
ax.plot(xx, gaussian(xx, result.params['totflux'] * result.params['scale'].value,
                     0,
                     result.params['narrow_x0'].value,
                     result.params['narrow_sigma'].value),
        lw=2, color='r', alpha=0.7, label='Narrow')
ax.plot(xx, gaussian(xx, result.params['totflux'] * (1 - result.params['scale'].value),
                     0.,
                     result.params['narrow_x0'].value,
                     result.params['broad_sigma'].value),
        lw=2, color='b', alpha=0.7, label='Broad')

ax.plot(xx, model.eval(result.params, x=xx),
        lw=2, color='k', label='Total', ls='--')

ax.legend()
ax.set_title(f'Module A+B')

plt.show()

In [ ]:
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

N_iter = 10000

MUV = np.random.normal(-21.35, 0.08, N_iter)
LUV_Lsun = 10**(0.4 * (4.85 - MUV))
LUV_ergs = LUV_Lsun * 3.846e33

scale_value = result.params['scale'].value
scale_stderr = result.params['scale'].stderr
scale = np.random.normal(scale_value, scale_stderr, N_iter)
# scale = 0.92

Hb_flux = result.params['totflux'].value * 1e-18
Hb_flux_err = result.params['totflux'].stderr * 1e-18

Dl = cosmo.luminosity_distance(6.5916).to(u.cm).value
LHB_BROAD = np.random.normal(Hb_flux, Hb_flux_err, N_iter) * 4*np.pi * Dl**2 * (1 - scale)

Lbolometric = 10.33 * 1e44 * pow(LHB_BROAD * 3.1 / 5.25e42, 1. / 1.157) #LHB_broad in erg/s
Lbol_to_UV_expected_Shen = Lbolometric/((1.862*(Lbolometric/3.846e43)**-0.361) + 4.87*(Lbolometric/3.846e43)**-0.0063)
MUV_expected = -np.log10(Lbol_to_UV_expected_Shen / 3.846e33) / 0.4 + 4.85
MUV_exp_percs = np.nanpercentile(MUV_expected, [16, 50, 84])
print('Expected MUV from broad component:', MUV_exp_percs[1], -MUV_exp_percs[1] + MUV_exp_percs)

print('L_UV_AGN / L_UV_total =', pow(10., (-21.35 - MUV_exp_percs[1])/2.5))

sigma_value = result.params['broad_sigma'].value
sigma_stderr = result.params['broad_sigma'].stderr
FWHM_Hb_wav = 2 * (2 * np.log(2))**0.5 * np.random.normal(sigma_value, sigma_stderr, N_iter)
FWHM_Hb_v = FWHM_Hb_wav / (4862.68 * (1 + this_z)) * 299.792458 # in 1e3 km/s
M_BH = 2.4e6 * (LHB_BROAD * 1e-42) ** 0.59 * FWHM_Hb_v**2. # in Msun
M_BH_percs = np.log10(np.nanpercentile(M_BH, [16, 50, 84]))
print('BH Mass:', M_BH_percs[1], (-M_BH_percs[1] + M_BH_percs))

In [ ]:
# FOR THE PAPER!!!!!!!!!!!!!!!!!!!!1
# scale = 0.93
scale_value = result.params['scale'].value
scale_stderr = result.params['scale'].stderr
scale = np.random.normal(scale_value, scale_stderr, N_iter)

Hb_flux = result.params['totflux'].value * 1e-18
Hb_flux_err = result.params['totflux'].stderr * 1e-18

Dl = cosmo.luminosity_distance(6.5916).to(u.cm).value
LHB_BROAD = np.random.normal(Hb_flux, Hb_flux_err, N_iter) * 4*np.pi * Dl**2 * (1 - scale)
Lbolometric = 10.33 * 1e44 * pow(LHB_BROAD * 3.1 / 5.25e42, 1. / 1.157) #LHB_broad in erg/s
print('L_bol =', np.log10(np.nanmedian(Lbolometric)))

sigma = 39.13
FWHM_Hb_wav = 2 * (2 * np.log(2))**0.5 * sigma
FWHM_Hb_v = FWHM_Hb_wav / (4862.68 * (1 + this_z)) * 299.792458 # in 1e3 km/s
M_BH = 2.4e6 * (LHB_BROAD * 1e-42) ** 0.59 * FWHM_Hb_v**2. # in Msun
M_BH_percs = np.log10(np.nanpercentile(M_BH, [16, 50, 84]))
print('BH Mass:', M_BH_percs[1], (-M_BH_percs[1] + M_BH_percs))

L_edd = 1.25e38 * 10**M_BH_percs[1]
print('L* =', np.log10(L_edd))
print('L_bol / L* =', np.nanmedian(Lbolometric / L_edd))

In [ ]:
FWHM_O3 = sigma * 2 * (2 * np.log(2))**0.5 / (5008.24 * (1 + this_z)) * 299792.458 # in km/s
print(FWHM_O3)
print(np.percentile(FWHM_Hb_v * 1000, [16, 50, 84]))

In [ ]:
# ANOTHER APPROACH
# Assume 100% of MUV comes from AGN: What's the expected M_BH?
LHB = np.random.normal(Hb_flux, Hb_flux_err, N_iter) * 4*np.pi * Dl**2

FWHM_Hb_wav = 2 * (2 * np.log(2))**0.5 * np.random.normal(sigma, sigma_err, N_iter)
FWHM_Hb_v = FWHM_Hb_wav / (4862.68 * (1 + this_z)) * 299.792458 # in 1e3 km/s
M_BH_full_AGN = 2.4e6 * (LHB * 1e-42) ** 0.59 * FWHM_Hb_v**2. # in Msun
M_BH_percs = np.log10(np.nanpercentile(M_BH_full_AGN, [16, 50, 84]))
print('BH Mass:', M_BH_percs[1], (-M_BH_percs[1] + M_BH_percs))

In [ ]:
from scipy.optimize import fsolve

def expected_MUV_solve(LHB_BROAD, norm=-21.35):
    Lbolometric = 10.33 * 1e44 * pow(LHB_BROAD * 3.1 / 5.25e42, 1. / 1.157) #LHB_broad in erg/s
    Lbol_to_UV_expected_Shen = Lbolometric/((1.862*(Lbolometric/3.846e43)**-0.361) + 4.87*(Lbolometric/3.846e43)**-0.0063)
    MUV_expected = -np.log10(Lbol_to_UV_expected_Shen / 3.846e33) / 0.4 + 4.85
    return MUV_expected - norm

sol = fsolve(expected_MUV_solve, 21)
LHB_pred_AGN = sol[0]
print('Predicted Hb =', np.log10(LHB_pred_AGN), 'Actual Hb =', np.log10(np.nanmedian(LHB)))
LHB_total = np.nanmedian(LHB)
print('Predicted scale = ', (LHB_total - LHB_pred_AGN) / LHB_total)

# FWHM_Hb as a function of M_BH
def M_BH_to_FWHM_Hb(M_BH, L_Hb):
    return (10.**M_BH / 2.4e6 * (L_Hb * 1e-42)**-0.59) ** 0.5 * 1e3

M_BH_to_FWHM_Hb(5.47, LHB_pred_AGN)

FWHM_Hb_wav = 2 * (2 * np.log(2))**0.5 * np.random.normal(sigma, sigma_err, N_iter)
FWHM_Hb_v = FWHM_Hb_wav / (4862.68 * (1 + this_z)) * 299.792458 # in 1e3 km/s
M_BH_full_AGN = 2.4e6 * (sol[0] * 1e-42) ** 0.59 * FWHM_Hb_v**2. # in Msun
M_BH_percs = np.log10(np.nanpercentile(M_BH_full_AGN, [16, 50, 84]))
print('BH Mass:', M_BH_percs[1], (-M_BH_percs[1] + M_BH_percs))

L_edd = 1.25e38 * 10**M_BH_percs[1]
print('L_edd =', L_edd)

Lbolometric = 10.33 * 1e44 * pow(LHB_pred_AGN * 3.1 / 5.25e42, 1. / 1.157) #LHB_broad in erg/s
# Lbol_to_UV_expected_Shen = Lbolometric/((1.862*(Lbolometric/3.846e43)**-0.361) + 4.87*(Lbolometric/3.846e43)**-0.0063)
print(f'L_bol = {Lbolometric / L_edd} L*')